In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
import gensim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.metrics import classification_report

from sklearn.metrics import classification_report, f1_score


#text is already cleaned.
#assign cleaned data to these variables.
train_data_path = 'cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = 'cleaned_data/cleaned_test_data_for_subtask1.csv'
#read files.
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

print("Train set:"% train_data.columns, train_data.shape, len(train_data)) 
print("Test set:"% test_data.columns, test_data.shape, len(test_data)) 



Train set: (20974, 8) 20974
Test set: (4997, 8) 4997


In [2]:
#prepare train and test data.
X_train = train_data['#2_tweet_clean_V2']
y_train = train_data['#classes_id']
X_test = test_data['#2_tweet_clean_V2']
y_test = test_data['#classes_id']

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_no = 50000
sql_len = 250
dim = 100

tokenizer = Tokenizer(num_words=max_no, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(X_train.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

tokenizer = Tokenizer(num_words=max_no, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(X_test.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 48685 unique tokens.
Found 17156 unique tokens.


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


X_tr = tokenizer.texts_to_sequences(X_train.values)
X_tr = pad_sequences(X_tr, maxlen=sql_len)
print('Shape of data tensor:', X_tr.shape)

X_te = tokenizer.texts_to_sequences(X_test.values)
X_te = pad_sequences(X_te, maxlen=sql_len)
print('Shape of data tensor:', X_te.shape)


Shape of data tensor: (20974, 250)
Shape of data tensor: (4997, 250)


In [5]:
Y_tr = pd.get_dummies(y_train).values
print('Shape of label tensor:', Y_tr.shape)

Y_te = pd.get_dummies(y_test).values
print('Shape of label tensor:', Y_te.shape)

Shape of label tensor: (20974, 21)
Shape of label tensor: (4997, 21)


In [7]:
!pip install keras

You are using pip version 19.0, however version 20.3.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
import keras
import tensorflow
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
#import tensorsflow_addons as tfa
model = keras.Sequential()
model.add(Embedding(max_no, dim, input_shape=(sql_len,)))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 20
batch_size = 64

history = model.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/20
295/295 [==============================] - 125s 416ms/step - loss: 2.6687 - accuracy: 0.2068 - val_loss: 2.4643 - val_accuracy: 0.2712
Epoch 2/20
295/295 [==============================] - 159s 539ms/step - loss: 2.2957 - accuracy: 0.3258 - val_loss: 2.2987 - val_accuracy: 0.3236
Epoch 3/20
295/295 [==============================] - 160s 543ms/step - loss: 1.9144 - accuracy: 0.4525 - val_loss: 2.3258 - val_accuracy: 0.3370
Epoch 4/20
295/295 [==============================] - 159s 539ms/step - loss: 1.6093 - accuracy: 0.5360 - val_loss: 2.4196 - val_accuracy: 0.3456
Epoch 5/20
295/295 [==============================] - 160s 543ms/step - loss: 1.3512 - accuracy: 0.6080 - val_loss: 2.6188 - val_accuracy: 0.3241
Epoch 6/20
295/295 [==============================] - 159s 539ms/step - loss: 1.1727 - accuracy: 0.6569 - val_loss: 2.7585 - val_accuracy: 0.3136
Epoch 7/20
295/295 [==============================] - 159s 540ms/step - loss: 1.0297 - accuracy: 0.6902 - val_loss: 2.8880 -

In [13]:
y_pred= model.predict(X_te)
y_pred = np.argmax(y_pred, axis=1)
y_test = test_data['#classes_id']
Y_te = pd.get_dummies(y_test).values

Y_te = np.argmax(Y_te, axis=1)
print(f1_score(Y_te, y_pred, average='macro'))


0.14653626397278355


In [15]:
target_names = list(train_data["#3_country_label"].unique())
print(classification_report(Y_te, y_pred,target_names=target_names))

                      precision    recall  f1-score   support

               Egypt       0.52      0.60      0.56      1041
                Iraq       0.38      0.41      0.39       663
        Saudi_Arabia       0.25      0.27      0.26       519
          Mauritania       0.20      0.17      0.18        53
             Algeria       0.38      0.34      0.36       430
               Syria       0.10      0.11      0.11       278
                Oman       0.17      0.15      0.16       355
             Tunisia       0.14      0.15      0.15       172
             Lebanon       0.10      0.09      0.09       157
             Morocco       0.08      0.09      0.08       207
            Djibouti       0.00      0.00      0.00        27
United_Arab_Emirates       0.08      0.08      0.08       157
              Kuwait       0.06      0.06      0.06       105
               Libya       0.28      0.23      0.25       314
             Bahrain       0.00      0.00      0.00        52
       